In [1]:
import os

In [2]:
%pwd

'd:\\project\\Face-to-BMI-Prediction\\research'

In [3]:
os.chdir("../")

In [4]:

%pwd

'd:\\project\\Face-to-BMI-Prediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path
from typing import List

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: List[int]
    params_batch_size: int
    params_include_top: bool
    params_epochs: int
    params_output_variables: List[dict]
    params_weights: str
    params_learning_rate: float


In [6]:
from BMI.constants import *
from BMI.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params["IMAGE_SIZE"],
            params_batch_size=self.params["BATCH_SIZE"],
            params_include_top=self.params["INCLUDE_TOP"],
            params_epochs=self.params["EPOCHS"],
            params_output_variables=self.params["OUTPUT_VARIABLES"],
            params_weights=self.params["WEIGHTS"],
            params_learning_rate=self.params["LEARNING_RATE"]
        )
        return prepare_base_model_config

In [8]:

import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

[2024-02-10 18:29:04,285: WARNING: module_wrapper: From d:\project\Face-to-BMI-Prediction\venv\lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]


In [9]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.applications.ResNet101V2(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)

    @staticmethod
    def _prepare_full_model(model, output_variables, learning_rate):
        flatten_in = tf.keras.layers.Flatten()(model.output)
        outputs = []
        for output_var in output_variables:
            if output_var["TYPE"] == "linear":
                outputs.append(tf.keras.layers.Dense(units=1, name=output_var["NAME"])(flatten_in))
            elif output_var["TYPE"] == "binary":
                outputs.append(tf.keras.layers.Dense(units=1, activation="sigmoid", name=output_var["NAME"])(flatten_in))

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=outputs
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss={
                output_var["NAME"]: "mean_squared_error" if output_var["TYPE"] == "linear" else "binary_crossentropy"
                for output_var in output_variables
            },
            metrics={output_var["NAME"]: "accuracy" for output_var in output_variables if output_var["TYPE"] == "binary"}
        )

        full_model.summary()
        return full_model
    

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            output_variables=self.config.params_output_variables,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


In [10]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2024-02-10 18:29:11,762: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-10 18:29:11,776: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-10 18:29:11,778: INFO: common: created directory at: artifacts]
[2024-02-10 18:29:11,779: INFO: common: created directory at: artifacts/prepare_base_model]
[2024-02-10 18:29:13,536: WARNING: module_wrapper: From d:\project\Face-to-BMI-Prediction\venv\lib\site-packages\keras\src\backend.py:1398: The name tf.executing_eagerly_outside_functions is deprecated. Please use tf.compat.v1.executing_eagerly_outside_functions instead.
]
[2024-02-10 18:29:15,636: WARNING: module_wrapper: From d:\project\Face-to-BMI-Prediction\venv\lib\site-packages\keras\src\layers\pooling\max_pooling2d.py:161: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.
]
[2024-02-10 18:29:23,261: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` w

d:\project\Face-to-BMI-Prediction\venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 pool1_pad (ZeroPadding2D)   (None, 114, 114, 64)         0         ['conv1_conv[0][0]']          
                                                                                              